In [1]:
import serial
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
for port in ports:
    print(port.device)

/dev/cu.debug-console
/dev/cu.wlan-debug
/dev/cu.usbserial-B0037O74
/dev/cu.Wall-E
/dev/cu.Maker-7644
/dev/cu.Maker3-5179
/dev/cu.Bluetooth-Incoming-Port


In [3]:
import serial
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
for port in ports:
    print(port.device)

with serial.Serial("/dev/cu.usbserial-B0037O74", 9600, timeout=1) as ser:
    while True:
        ser.write(b"R\r\n")
        response = ser.readline() 
        print(response)

/dev/cu.debug-console
/dev/cu.wlan-debug
/dev/cu.usbserial-B0037O74
/dev/cu.Wall-E
/dev/cu.Maker-7644
/dev/cu.Maker3-5179
/dev/cu.Bluetooth-Incoming-Port
b'?CAL,0\r?U,PSI\r2.659\r'
b'2.659\r'
b'2.658\r'
b'2.659\r'
b'2.660\r'
b'2.659\r'
b'2.660\r'
b'2.659\r'
b'2.659\r'
b'2.659\r'
b'2.659\r'
b'2.660\r'
b'2.658\r'
b'2.659\r'
b'2.659\r'


SerialException: read failed: [Errno 6] Device not configured

# Serial backup

In [ ]:
#Pressure

import serial
import serial.tools.list_ports
import csv
import time

#ports = serial.tools.list_ports.comports()
#for port in ports:
#    print(port.device)


serial_port = "/dev/cu.usbserial-B0037O74"
baud_rate = 9600

timestamp_format = '%Y%m%dT%H%M%S.%f'
output_file = "pressure_log_"+str(now.strftime('%Y%m%dT%H%M%S.%f')+".csv")

with serial.Serial(serial_port, baud_rate, timeout=1) as ser, open(output_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "Pressure(psi)"])
    
    while True:
        ser.write(b"R\r\n") 
        response = ser.readline().decode().strip()
        timestamp = time.strftime(timestamp_format)
        writer.writerow([timestamp, response])
        print(timestamp, response)

In [ ]:
#Hum

import serial
import serial.tools.list_ports
import csv
import time

#ports = serial.tools.list_ports.comports()
#for port in ports:
#    print(port.device)


serial_port = "/dev/cu.usbserial-B0037O74"
baud_rate = 9600

timestamp_format = '%Y%m%dT%H%M%S.%f'
output_file = "humidity_log_"+str(now.strftime('%Y%m%dT%H%M%S.%f')+".csv")

with serial.Serial(serial_port, baud_rate, timeout=1) as ser, open(output_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "Hum","Tmp","Dew"])
    
    while True:
        ser.write(b"R\r\n") 
        response = ser.readline().decode().strip(). split(',')
        hum = float(response[0])
        tmp = float(response[1])
        dew = float(response[3])
        timestamp = time.strftime(timestamp_format)
        writer.writerow([timestamp, hum, tmp, dew])
        print(timestamp, hum, tmp, dew)

In [ ]:
#temperatures

#Ver 2
import numpy as np
import serial
import time
from datetime import datetime as dt
import csv

def open_serial(serial_port):
    baud_rate = 38400
    ser = serial.Serial(serial_port, baud_rate, timeout=1)
    return ser
    
def hex_dec(T_hex):
    try:
        T_val = int(T_hex, 16)
        T_max = 18000  # 1800C max
        hex_max = 0xFFFF  # FFFF max 
        if T_val > T_max: 
            T = -(hex_max - T_val + 1) / 10 ##negative value
        else:
            T = T_val / 10
        return T
    except ValueError:
        return 'err'

def parse_temp(response):
    response_hex = response.hex()
    temperatures = []

    for i in range(8):
        hex_str = response_hex[34 + i*4:36 + i*4] + response_hex[32 + i*4:34 + i*4]
        temperatures.append(hex_dec(hex_str))

    return tuple(temperatures)

def readtemp(ser, byte_data):
    ser.write(byte_data)

    response = ser.readline()
    while True:
        ser.write(byte_data)
        time.sleep(0.1) 
        if ser.in_waiting == 37:
            response = ser.readline()
            if response:
                try:
                    return parse_temp(response)
                except ValueError:
                    return ('err',) * 8 #if not valid hex, put err
            else:
                return (np.nan,) * 8 #if no response, put nan
    
#read temp
ser = open_serial('/dev/cu.usbserial-110') 
hex_data = [0x01, 0x16, 0x7B, 0x28, 0x48, 0x4C, 0x45, 0x48, 0x54, 0x43, 0x34, 0x30, 0x39, 0x35, 0x67, 0x71, 0x29, 0x7D, 0x7E, 0x04]
byte_data = bytearray(hex_data)

now = dt.now()
filename = "temp_log_"+str(now.strftime('%Y%m%dT%H%M%S.%f')+".csv")
print("starting temperature log file: "+filename)

with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['DateTimestamp','T1','T2','T3','T4','T5','T6','T7','T8'])

    while True:
        now = dt.now()
        nowstr = now.strftime('%Y%m%dT%H%M%S.%f')
        T1,T2,T3,T4,T5,T6,T7,T8 = readtemp(ser, byte_data) 
        print(f"{nowstr},{T1},{T2},{T3},{T4},{T5},{T6},{T7},{T8}")
        csvwriter.writerow([nowstr,T1,T2,T3,T4,T5,T6,T7,T8])
        csvfile.flush()
    csvfile.close()     
    ser.close()